## Notebook to practice inference & clustering & stats analysis 

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import glob
import numpy as np
import scipy as sc
import pandas as pd
import tensorflow as tf
from pyhdf.SD import SD, SDC 

import matplotlib.pyplot as plt
from matplotlib import patches as mpl_patches
from sklearn.cluster import AgglomerativeClustering

---------------
### Prerequisites
Libraries below are necessary to download on you laptop (If disk amount is issue, we can move to Midway cluster-computer where the analysis is much easier if you have already got the access)

1. Tensorflow 1.12.0 for CPU [stackoverflow How to install CPU version of Tnesorflow](https://stackoverflow.com/questions/53614262/how-to-install-cpu-version-of-tensorflow-using-conda)
2. Agglometative clustering from scikit-learn.  
 How to donwload agglomerative [sklearn aggl](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)  

`from sklearn.cluster import AgglomerativeClustering`

### Helpful resource
- `clouds/analysis_mod021KM/017_m2_02_clustering-stats_with-without_DNNprocessing.ipynb`
- `clouds/analysis_mod021KM/016_m2_02_clustering_with-without_CloudFlags.ipynb`  
These notebooks contain lot of original function/analysis (especially 017)

### Load a trained DNN model

In [ ]:
# model directory path
modeldir = ""
step = 100000 # DO NOT CHANGE so far

In [ ]:
encoder_def = modeldir+'/encoder.json'
encoder_weight = modeldir+'/encoder-'+str(step)+'.h5'
with open(encoder_def, "r") as f:
    encoder = tf.keras.models.model_from_json(f.read())
encoder.load_weights(encoder_weight)

### Data description

### Load MODIS 02, 35 data below
- Try to code by yourself based on practice notebook (firstly look into notebooks, and then if anything questions you havev, ask Takuya how to do this.)

### Clustering
1. Start at N = 12  (N is number of clusters)
2. Next try N = 48
3. Next vary the N. Apply N = 4, 8, 12, ... 52 (every4) or 4,6,8,...,50(every2)

### Stats analysis
At first from N = 12 case.
0. Compute mean value of each patch with respect to 4 clouds physics parameters as follwoing
 - optical thickness
 - water path
 - particle phase
 - top pressure
 
1. compute distributions, mean, stdv of these clouds physics parameters 

2. Plot histogram about the distribution. Add mean and std information on these figures